In [1]:
import pandas as pd
import numpy as np
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from google.colab import drive
drive.mount('/gdrive')



Mounted at /gdrive


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.5 MB/s eta 0:00:00


In [4]:
from transformers import pipeline

In [5]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_analysis = pipeline("sentiment-analysis", model = model_name, device = 0, truncation = True)
print(sentiment_analysis("I love this !"))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'label': 'positive', 'score': 0.977944016456604}]


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

In [7]:
%cd /gdrive/MyDrive/qlik/escalation_project/train_9_27

/gdrive/MyDrive/qlik/escalation_project/train_9_27


In [8]:
def get_label_val(s):
  if s == 'negative':
    return 0
  if s == 'neutral':
    return 1
  return 2


In [9]:
%pwd

'/gdrive/MyDrive/qlik/escalation_project/train_9_27'

In [10]:
train_df = pd.read_csv("test.csv")
train_df.shape

(58000, 31)

In [11]:
from tqdm import tqdm
scores = []

temp = train_df['COMMENT_BODY'].tolist()
for temp_str in tqdm(temp):
  scores.append(sentiment_analysis(temp_str, truncation = True, padding=True, max_length = 512))

  0%|          | 6/58000 [00:00<58:51, 16.42it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 58000/58000 [17:16<00:00, 55.96it/s]


In [12]:
labels =[x[0]['label'] for x in scores]
confidence_scores =[x[0]['score'] for x in scores]

In [13]:
label_vals = list(map(get_label_val, labels))

In [14]:
sentiment_score = np.array(label_vals).astype(np.float) * 1/3 + 1/3 * np.array(confidence_scores).astype(np.float)

<ipython-input-14-4252bd7efd8e>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sentiment_score = np.array(label_vals).astype(np.float) * 1/3 + 1/3 * np.array(confidence_scores).astype(np.float)


In [15]:
train_df['sentiment_label'] = labels
train_df['sentiment_confidence_score'] = confidence_scores
train_df['sentiment_score'] = sentiment_score

In [16]:
train_df.to_csv("test_sentiment.csv")